# 1. Prepare files 

In [ ]:

import pandas as pd
import os
import re
import pyarrow.parquet as pq
import pyarrow
import fastparquet
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", None)

def clean_data(df):
    #Renaming
    df.index.name = 'Datetime'
    df.rename(columns={'SET_TEMP': 'Set_temp', 'HVAC_STATE': 'Hvac_state','ROOM_OCC': 'Room_occupation','WINDOW': 'Window',
                       'MODE': 'Hvac_mode','MANUAL': 'Hvac_state_manual','TEMP_UP': 'Room_temp_up','TEMP_DOWN': 'Room_temp_down',
                       'TEMP_CW': 'Room_temp_cw','TEMP_CCW': 'Room_temp_ccw','T_AIR_ANOM': 'Room_temp','FAN_SPEED_ANOM': 'Fan_speed',
                       'ENERGY_ANOM': 'Energy_consumption','RH': 'Humidity','T_WATER': 'Water_temp','ENV_TEMP': 'Outside_temp',
                       'IRRAD': 'Irradiation','DETECTED_ANOM': 'is_detected'}, inplace=True)
    #One Hot Encoding
    df.loc[df['Fan_speed'] ==0, 'FS_0'] = 1
    df.loc[df['Fan_speed'] ==1, 'FS_1'] = 1
    df.loc[df['Fan_speed'] ==2, 'FS_2'] = 1
    df.loc[df['Fan_speed'] ==3, 'FS_3'] = 1
    df.loc[df['Fan_speed'] >0, 'FS_0'] = 0
    df.loc[df['Fan_speed'] !=1, 'FS_1'] = 0
    df.loc[df['Fan_speed'] !=2, 'FS_2'] = 0
    df.loc[df['Fan_speed'] !=3, 'FS_3'] = 0
    #Removing
    
    df.drop(columns=['Fan_speed','FAN_SPEED_GOOD', 'T_AIR_GOOD','ENERGY_GOOD','IRRAD_ANG','INSERT_valve', 
                     'INSERT_fan1','INSERT_fanStop','INSERT_win'], inplace=True)
    return df

In [3]:
fault='F11'

South = '.*_0_.*'
West = '.*_90_.*'
North = '.*_180_.*'
East = '.*_270_.*'

Y1 = '.*Y2015.*'
Y2 = '.*Y2016.*'
Y3 = '.*Y2017.*'
Y4 = '.*Y2018.*'
Y5 = '.*Y2019.*'
Y6 = '.*Y2021.*'


path=f'C:/Users/imate/Documents/Data/Simulations/{fault}'
parquet_files = [f for f in os.listdir(path) if f.endswith('.parquet')]

for i, parquet_file in enumerate(parquet_files):
    df = pd.read_parquet(os.path.join(path, parquet_file))
    var = re.search("(Y\d{4}_R\d+)", parquet_file).group(1)
    if re.match(South, parquet_file):
        df['Orientation_S'] = 1
        df['Orientation_W'] = 0
        df['Orientation_N'] = 0
        df['Orientation_E'] = 0
    elif re.match(West, parquet_file):
        df['Orientation_S'] = 0
        df['Orientation_W'] = 1
        df['Orientation_N'] = 0
        df['Orientation_E'] = 0
    elif re.match(North, parquet_file):
        df['Orientation_S'] = 0
        df['Orientation_W'] = 0
        df['Orientation_N'] = 1
        df['Orientation_E'] = 0
    elif re.match(East, parquet_file):
        df['Orientation_S'] = 0
        df['Orientation_W'] = 0
        df['Orientation_N'] = 0
        df['Orientation_E'] = 1
        
    #CLEAN DATA
    df_cleaned= clean_data(df) 
    globals()[f'df_'+var] = df_cleaned
    x=globals()[f'df_'+var]
    x.to_csv(f'C:/Users/imate/Documents/Data/Rooms/{fault}/df_'+var+'.csv')

# 2. Merge 

In [ ]:
fault='F11'
path = f'C:/Users/imate/Documents/Data/Rooms/{fault}'
files = [f for f in os.listdir(path) if f.endswith('.csv')]

Y1 = '.*_Y2015_*'
Y2 = '.*_Y2016_*'
Y3 = '.*_Y2017_*'
Y4 = '.*_Y2018_*'
Y5 = '.*_Y2019_*'
Y6 = '.*_Y2021_*'

l_15=[]
l_16=[]
l_17=[]
l_18=[]
l_19=[]
l_21=[]

for i, file in enumerate(files):
    df = pd.read_csv(os.path.join(path, file), engine='python',skipfooter=1)
    if re.match(Y1, file):
        l_15.append(df)
    elif re.match(Y2, file):
        l_16.append(df)
    elif re.match(Y3, file):
        l_17.append(df)
    elif re.match(Y4, file):
        l_18.append(df)
    elif re.match(Y5, file):
        l_19.append(df)
    elif re.match(Y6, file):
        l_21.append(df)


In [4]:
ll_15=pd.concat(l_15)
ll_16=pd.concat(l_16)
ll_17=pd.concat(l_17)
ll_18=pd.concat(l_18)
ll_19=pd.concat(l_19)
ll_21=pd.concat(l_21)

In [5]:
ll_15.to_csv(f'C:/Users/imate/Documents/Data/Merge/{fault}/Y2015.csv')
ll_16.to_csv(f'C:/Users/imate/Documents/Data/Merge/{fault}/Y2016.csv')
ll_17.to_csv(f'C:/Users/imate/Documents/Data/Merge/{fault}/Y2017.csv')
ll_18.to_csv(f'C:/Users/imate/Documents/Data/Merge/{fault}/Y2018.csv')
ll_19.to_csv(f'C:/Users/imate/Documents/Data/Merge/{fault}/Y2019.csv')
ll_21.to_csv(f'C:/Users/imate/Documents/Data/Merge/{fault}/Y2021.csv')